In [8]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [1]:
!ls
!nvidia-smi

sample_data
Wed Dec 24 09:17:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [2]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 336 (delta 125), reused 126 (delta 60), pack-reused 138 (from 1)
Receiving objects: 100% (336/336), 98.13 MiB | 16.42 MiB/s, done.
Resolving deltas: 100% (194/194), done.
/content/no-time-to-backprop


In [3]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-linux-cuda.txt  run_all_toy.sh
hpt.py				 requirements-macos-cpu.txt   run_on_gpu.ipynb
plotting_scripts		 requirements.mac.txt	      run_toy_task.py
requirements-base.txt		 requirements.txt	      test_rec.py
requirements.from_linux_env.txt  run_all_benchmarks.sh	      tgap


In [4]:
%pip install -r requirements-linux-cuda.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 139.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 22.3 MB/s eta 0:00:00
   ━━━━

In [5]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [6]:
# Optuna trials
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --batching_strategy rearranged --num_epochs 100 --dedupe --hpt optuna --hpt_config ./tgap/benchmarks/hpt_config_example.json


[*] Results will be stored in /content/no-time-to-backprop/results_for_benchmarks/mooc_ONLINE
[*] d_model set to 32
[I 2025-12-24 09:19:11,941] A new study created in memory with name: no-name-e7cefebb-a827-4f94-8fd4-d3c2f12dbb06

[*] ----------------------------- Iteration: 0 -----------------------------

[*] Trial: <optuna.trial._trial.Trial object at 0x7df878b819d0>
[*] HPT: {'memory': 5, 'seed': 43, 'num_layers': 5, 'state_size': 32, 'd_model_factor': 0.5, 'dropout': 0.26759833212610545, 'pos_weight': 1.4030429932415482, 'batch_size': 64, 'n_accumulation_steps': 2, 'learning_rate': 0.001636436813352925, 'rec_learning_factor': 0.5, 'beta1': 0.915908070235828, 'beta2': 0.9650544716257586, 'weight_decay': 0.0001096021655388051}
[*] Npz file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists with matching meta. Using existing file.
W1224 09:19:12.282480     759 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support ex

In [ ]:
!ls

sample_data
